# State Feedback
In this notebook, we aim to learn the concept of state feedback. First, we visualaize this concept by working on a the model of a simple car. Then, we define the state feedback mathematically. Below we import some libraries and define the ploter:

In [2]:
#@title
import numpy as np
import scipy.integrate as integrate
from IPython import display
from IPython.display import HTML
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from collections import deque


def plot_simple_car(trajectory, par):
    t = np.arange(0, par['t_stop'], par['dt'])  # create a time array from 0..t_stop sampled at 0.02 second steps
    history_len = int(par['t_stop'] / par['dt'])  # how many trajectory points to display

    x1 = trajectory[:, 0]
    y1 = trajectory[:, 2]
    
    plt.style.use('seaborn')
    fig = plt.figure(figsize=(10, 4))
    gs = gridspec.GridSpec(2, 2)
    
    ax1 = fig.add_subplot(gs[:, 0], autoscale_on=False, xlim=(-5.0, 5.0), ylim=(-5.0, 5.0))
    ax1.set_aspect('equal')
    ax1.grid()
    ax1.set_xlabel('x')
    ax1.set_ylabel('y')
    ax1.annotate('Target', xy=(0, 0), xycoords='data', xytext=(35, 5), textcoords='offset points',
                 horizontalalignment='right', verticalalignment='bottom')

    line, = ax1.plot(0, 0, '+', ms=15.0, mec='k')
    line, = ax1.plot([], 'o-', lw=2, c='r')
    trace, = ax1.plot([], ',-', lw=1, c='b')
    time_template = 'time = %.1fs'
    time_text = ax1.text(0.05, 0.9, '', transform=ax1.transAxes)
    history_x, history_y, history_t = deque(maxlen=history_len), deque(maxlen=history_len), deque(maxlen=history_len)

    ax2 = fig.add_subplot(gs[0, 1], autoscale_on=False, xlim=(0.0, par['t_stop']), ylim=(-5.0, 5.0))
    ax2.grid()
    ax2.set_xlabel('time')
    ax2.set_ylabel('x')
    line2, = ax2.plot([], 'o-', lw=2, c='r')
    trace2, = ax2.plot([], ',-', lw=1, c='b')
    

    ax3 = fig.add_subplot(gs[1, 1], autoscale_on=False, xlim=(0.0, par['t_stop']), ylim=(-5.0, 5.0))
    ax3.grid()
    ax3.set_xlabel('time')
    ax3.set_ylabel('y')
    line3, = ax3.plot([], 'o-', lw=2, c='r')
    trace3, = ax3.plot([], ',-', lw=1, c='b')
    
    def animate(i):
        thisx = [x1[i]]
        thisy = [y1[i]]
        thist = i * par['dt']

        if i == 0:
            history_x.clear()
            history_y.clear()
            history_t.clear()

        history_x.appendleft(thisx[0])
        history_y.appendleft(thisy[0])
        history_t.appendleft(thist)

        line.set_data(thisx, thisy)
        trace.set_data(history_x, history_y)
        time_text.set_text(time_template % (i * par['dt']))

        line2.set_data(thist, thisx)
        trace2.set_data(history_t, history_x)

        line3.set_data(thist, thisy)
        trace3.set_data(history_t, history_y)

        return line, trace, time_text, line2, trace2, line3, trace3
    
    print("🧛 animating :) please wait...")
    fig.suptitle(par['title'], fontsize='xx-large')
    plt.show()
    ani = animation.FuncAnimation(fig, animate, len(trajectory), interval=par['dt']*1000, blit=True)
    display.clear_output(wait=True)
    return HTML(ani.to_html5_video())
    
    

# State Feedback
In this notebook, we aim to learn the concept of state feedback. First, we visualaize this concept by working on a the model of a simple car. Then, we define sthe state feedback mathematically.


## A simple car model
We model a simple car by a double integrator in x and y coordinates. Here is the model in the x coordinate
\begin{align*}
&\dot{p}_{x}=v_{x}\\
&\dot{v}_{x}=u_{x}
\end{align*}

where $p_x$ denotes the position in the x coordinate, $v_x$ is the velocity in the x coordinate and $u_x$ is the acceleration we can give to our car in the x coordinate. The dynamics in the y coordinate is defined similarly
\begin{align*}
&\dot{p}_{y}=v_{y}\\
&\dot{v}_{y}=u_{y}
\end{align*}

The car starts from some nonzero initial positions and velocities and the aim is to bring it to position $(0,0)$ with zero velocities. Let's build this model:


In [3]:
def simple_car(par):
    # Double Integrator Dynamics
    A = np.array([[0.0, 1.0], [0.0, 0.0]])
    B = np.array([[0.0], [1.0]])
    Kx = par["Kx"]
    Ky = par["Ky"]
    # integrate your ODE using scipy.integrate

    def derivs(state, t):
        state_x = (state[0: 2]).reshape((2, 1))
        state_y = (state[2: 4]).reshape((2, 1))
        dxdt = (A - B @ Kx) @ state_x
        dydt = (A - B @ Ky) @ state_y
        dstate_dt = np.concatenate([dxdt, dydt], axis=0)
        return dstate_dt.flatten()

    trajectory = integrate.odeint(derivs, par["init_state"], np.arange(0, par['t_stop'], par['dt']))
    return trajectory

First, let's see how the car moves when no acceleration is given

In [4]:
#@title
no_controller = {
    'init_state': np.array([-3.0, 1.0, 2.0, 1.0]),
    'Kx': np.array([[0.0, 0.0]]),
    'Ky': np.array([[0.0, 0.0]]),
    't_stop': 5,  # how many seconds to simulate
    'dt': 0.02,
    'title': 'No controller for the car'
}
trajectory_no_controller = simple_car(no_controller)
plot_simple_car(trajectory_no_controller, no_controller)

You can see that the car moves with constant velocity according to its initial valocities in x and y directions.

Now, lets design the acceleration. My suggestion is
\begin{align*}
u_x = -4 p_x -5 v_x,\\
u_y = -4 p_y -5 v_y.
\end{align*}
This controllers try to select the accerlerations in the opposite directions of the current position and velocity; i.e. to bring to zero. Feel free to change the controller gain to other positive values, the initial condition of the car and the simulation time to explore what you like

In [5]:
#@title Feedback controller parameters { run: "auto" }
#@markdown In x direction
position_gain_x = 4.0 #@param {type:"slider", min:1.0, max:10.0, step:0.5}
velocity_gain_x = 5.0 #@param {type:"slider", min:1.0, max:10.0, step:0.5}

#@markdown In y direction
position_gain_y = 4.0 #@param {type:"slider", min:1.0, max:10.0, step:0.5}
velocity_gain_y = 5.0 #@param {type:"slider", min:1.0, max:10.0, step:0.5}


your_controller = {
    'init_state': np.array([-3.0, 1.0, 2.0, 1.0]),
    'Kx': np.array([[position_gain_x, velocity_gain_x]]),
    'Ky': np.array([[position_gain_y, velocity_gain_y]]),
    't_stop': 5,  # how many seconds to simulate
    'dt': 0.02,
    'title': 'A state feedback controller'
}
trajectory = simple_car(your_controller)
plot_simple_car(trajectory, your_controller)

The controller we just implmented can be seen as a PD controller
* The P term: We feed in a proportional of the position  which corresponds to the P term 
* The D term: We also feed in a proportional of the position derivative; i.e. a proportional of the velocity which corresponds to the D term.
Now let's see if using a P term or a D term is enough for controlling the system. First we consider a P controller. That is we set all gains related to the velocity equal to zero.

In [6]:
#@title A P controller: The D gain is set to zero { run: "auto" }
#@markdown In x direction
position_gain_x = 4.0 #@param {type:"slider", min:1.0, max:10.0, step:0.5}

#@markdown In y direction
position_gain_y = 4.0 #@param {type:"slider", min:1.0, max:10.0, step:0.5}


P_controller = {
    'init_state': np.array([-3.0, 1.0, 2.0, 1.0]),
    'Kx': np.array([[position_gain_x, 0.0]]),
    'Ky': np.array([[position_gain_y, 0.0]]),
    't_stop': 5,  # how many seconds to simulate
    'dt': 0.02,
    'title': 'A proportional (P) controller'
}

trajectory_P = simple_car(P_controller)
plot_simple_car(trajectory_P, P_controller)

You can see that it is not possible to control the system by using only the position. The reason is that if the controller is only proportional to the position, when the position is zero or the car is in target, no control is applied to the system. However, the system will not stay at the target position because the velocity is non zero which results the car moves from the target position. This results an oscillatory behavior as you can see in the plot above.

Now, we go ahead and apply only a D controller to the system. That is we set all gains related to the position equal to zero.

In [7]:
#@title A D controller: The P gain is set to zero { run: "auto" }
#@markdown In x direction
velocity_gain_x = 5.0 #@param {type:"slider", min:1.0, max:10.0, step:0.5}

#@markdown In y direction
velocity_gain_y = 5.0 #@param {type:"slider", min:1.0, max:10.0, step:0.5}

D_controller = {
    'init_state': np.array([-3.0, 1.0, 2.0, 1.0]),
    'Kx': np.array([[0, velocity_gain_x]]),
    'Ky': np.array([[0, velocity_gain_y]]),
    't_stop': 5,  # how many seconds to simulate
    'dt': 0.02,
    'title': 'A derivative (D) controller'
}

trajectory_D = simple_car(D_controller)
plot_simple_car(trajectory_D, D_controller)

Again you can see that using a D controller is not enough. Indeed the controller is only proportional to the velocity. That is when the velocity is zero, the control signal is zero and there is no way of correcting the car position. All control effort is directed towards zeroing the velocity only. This shows we need both the position and the velocity in our controller.

What we discussed here is very ad-hoc. We cannot reason about other problems in this way and design the controller. We need to have a mathematical vision towards this problem. 

## State feedback
As we saw, we cannot achieve our control objective by feeding only the output (position in our example). In this section, we define the concept of state feedback. First let's replace the car with a more general linear dynamics
\begin{align*}
\dot{x}=A x +B u
\end{align*}
where $x \in \mathbf{R}^{n}$ represents the state, $u \in \mathbf{R}^{m}$ represents the control input (which we will design), $A$ is called the internal dynamics and $B$ is called the input matrix. We are interested in controlling the above dynamical system to achieve a desired behavior. From now on, we assume that the state variable is available for the controller design. We will cover in another notebook why and when this assumtion is valid.

If no control is applied to the system ($u \equiv 0$), the dynamical system evolves according to internal dynamics $A$
\begin{align*}
\dot{x}= A x
\end{align*}
So the behavior of the system is defined by the matrix $A$ and mostly by its eigenvalues.

However, if we select the controller as $u=-K x$ we can affect the internal dynamics change it to something else (within some limits imposed by the dynamics)
\begin{align*}
\dot{x}=& A x+B u\\
=&Ax-BKx=(A-BK)x
\end{align*}
Now, the dynamical systems evolves according to its new internal dynamics $A_{cl}=A-BK$. We can select the matrix $K$ to achieve a desired objective, for example to have stability. To do so it is enough to select $K$ such that the new internal dynamics $A_{cl}=(A-BK)$ has negative eigenvalues

Lets comeback and revisit our probelm. We suggested to use a state feedback containing both the position and velocity gains for the simple car. Let's see the eigenvalues of $A-BK$ 

In [8]:
A = np.array([[0.0, 1.0], [0.0, 0.0]])
B = np.array([[0.0], [1.0]])
print(np.linalg.eig(A-B@your_controller['Kx'])[0])

[-1. -4.]


You can see that both eigenvalues are negative and as such the simple car dynamics is stable. Now let's see why two other choices of the controller gain do not work. We will undersand it by checking the eigenvalues. First the P case when we have the position gain only:


In [9]:
print(np.linalg.eig(A-B@P_controller['Kx'])[0])

[0.+2.j 0.-2.j]


You can see that the eigenvalues have zero real parts which result in an oscillatory behavior. Now, we check the D case, when we have the velocity gain only:

In [10]:
print(np.linalg.eig(A-B@D_controller['Kx'])[0])

[ 0. -5.]


While one of the eigenvalues is negative (stable) the other one is zero and again the system is not internally stable.